In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Okt
import pickle

In [ ]:
pd.set_option('display.unicode.east_asian_width', True)

In [ ]:
raw_df = pd.read_csv('./data/raw_data_reduced.csv', index_col=0)
raw_df.head(10)

,Title,Medium_category,Small_category,Introduction,cnt
1,-10KG 밀가루 단식,가정 살림,요리,내 몸 리셋 다이어트 10kg 밀가루 단식 저자의 100일 밀가루 단식 다이어...,1
2,디디미니의 초간단 인생맛 고단백 저탄수화물 다이어트 레시피,가정 살림,요리,맛있고 배부른데 살까지 빠지는 다이어트 레시피 끝판왕 101가지 2018년...,1
3,맛있게 쓴 옥주부 레시피 100,가정 살림,요리,옥주부가 매일 자신의 인스타그램에 밥상 메뉴를 올리는 이유는 뭐 먹고 사는지 ...,1
4,세상 쉽고 맛있는 튼이 이유식,가정 살림,요리,쌀가루 큐브 밥솥 칸막이를 활용한 최초의 이유식 책 엄마의 정성과 사랑...,1
5,맛있게 살 빠지는 고단백 저탄수화물 다이어트 레시피,가정 살림,요리,다이어트 할 때도 맛있게 먹어야 성공한다 미니는 22kg 엄마는 17kg을 ...,1
6,임지호의 밥 땅으로부터,가정 살림,요리,면역력이 시대의 슬로건이 된 이때 모두에게 추천하는 임지호의 들풀밥상 ...,1
7,유아식 레시피북,가정 살림,요리,아이가 잘 안 먹는 것은 엄마 잘못이 아니에요 돌부터 요리를 시작하는 ...,1
8,와인이 있는 100가지 장면,가정 살림,요리,라라랜드 에서 미아가 꿈속에서 마시는 와인은 뭘까 아이언맨 의 토니 스타...,1
9,백종원이 추천하는 집밥 메뉴 애장판,가정 살림,요리,국민 요리책 백종원이 추천하는 집밥 메뉴 시리즈 합본 한정판 1 4탄 백...,1
10,[예스리커버] 매직 레시피,가정 살림,요리,난생처음 요리하는 당신이라도 전문점 맛 그대로 YouTube 요리 ...,1


In [ ]:
print('initial:', len(raw_df))

initial: 41170


## data 공백 처리

In [ ]:
for i in range(len(raw_df)):
    for j in range(5, 1, -1):
        raw_df.iloc[i,3] = raw_df.iloc[i,3].replace(' '*j,  ' ')

In [ ]:
print('after gap-healing:', len(raw_df))

after gap-healing: 41170


## data 중복 제거

In [ ]:
# null값 확인
# raw_df['Introduction'].isnull().values.any()

In [ ]:
# 중복된 data 개수 확인
sum_dup = raw_df.Introduction.duplicated().sum()
sum_dup

184

In [ ]:
# 중복된 data 제거(row)
df = raw_df.drop_duplicates(subset=['Introduction'])
sum_dup = df.Introduction.duplicated().sum()
sum_dup

0

In [ ]:
print('after dropna', len(df))

after dropna 40986


In [ ]:
# 새로운 index 할당
df.reset_index(drop=True, inplace=True)  # drop=True : 기존 index를 제거
df

,Title,Medium_category,Small_category,Introduction,cnt
0,-10KG 밀가루 단식,가정 살림,요리,내 몸 리셋 다이어트 10kg 밀가루 단식 저자의 100일 밀가루 단식 다이어트 ...,1
1,디디미니의 초간단 인생맛 고단백 저탄수화물 다이어트 레시피 ...,가정 살림,요리,맛있고 배부른데 살까지 빠지는 다이어트 레시피 끝판왕 101가지 2018년 201...,1
2,맛있게 쓴 옥주부 레시피 100,가정 살림,요리,옥주부가 매일 자신의 인스타그램에 밥상 메뉴를 올리는 이유는 뭐 먹고 사는지 자랑...,1
3,세상 쉽고 맛있는 튼이 이유식,가정 살림,요리,쌀가루 큐브 밥솥 칸막이를 활용한 최초의 이유식 책 엄마의 정성과 사랑이 듬뿍 담...,1
4,맛있게 살 빠지는 고단백 저탄수화물 다이어트 레시피...,가정 살림,요리,다이어트 할 때도 맛있게 먹어야 성공한다 미니는 22kg 엄마는 17kg을 감량하...,1
...,...,...,...,...,...
40981,한국인의 초보 사주팔자,인문,명리/주역/풍수,인생 쉽지 않다 생각한 대로 흘러가지 않기 때문이다 지성과 이성으로 내린 선택과 ...,1
40982,다르게 살고 싶다,인문,명리/주역/풍수,나는 어떤 사람인가 타인의 욕망을 좇는 삶에서 자기 주도적인 삶으로 이 책은 스펙...,1
40983,주역과 만나다 하,인문,명리/주역/풍수,주역 은 한마디로 변화 의 책이다 변화는 밤하늘을 화려하게 수놓는 일월성신의 규칙...,1
40984,주역과 만나다 중,인문,명리/주역/풍수,주역 은 하늘과 땅 해와 달 바람과 우레 뫼와 연못이라는 여덟 가지 형상을 우주의...,1


In [ ]:
# 12개의 category
Medium_ctg = df['cnt'].groupby(df['Medium_category'])
Medium_ctg.sum() 

Medium_category
가정 살림           2494
건강 취미           3156
경제 경영           3607
국어 외국어 사전    2496
만화/라이트노벨     5694
사회 정치           2858
소설/시/희곡        4774
수험서 자격증       3716
어린이              3806
유아                3084
인문                2720
청소년              2581
Name: cnt, dtype: int64

## category 병합

In [ ]:
columns = list(df.columns)
columns

['Title', 'Medium_category', 'Small_category', 'Introduction', 'cnt']

In [ ]:
# 12개 -> 6개 category로 병합
df['Large_category'] = 0

df.loc[(df['Medium_category'] == '가정 살림') | (df['Medium_category'] == '건강 취미'), 'Large_category'] = '가정/건강'
df.loc[(df['Medium_category'] == '인문') | (df['Medium_category'] == '사회 정치') | (df['Medium_category'] == '경제 경영'), 'Large_category'] = '인문/사회/경영'
df.loc[(df['Medium_category'] == '국어 외국어 사전') | (df['Medium_category'] == '수험서 자격증'), 'Large_category'] = '사전/자격증'
df.loc[(df['Medium_category'] == '만화/라이트노벨'), 'Large_category'] = '만화'
df.loc[(df['Medium_category'] == '소설/시/희곡'), 'Large_category'] = '소설/시/희곡'
df.loc[(df['Medium_category'] == '어린이') | (df['Medium_category'] == '유아') | (df['Medium_category'] == '청소년'), 'Large_category'] = '어린이/청소년'


ipykernel_launcher:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/kaeun/.conda/envs/tf/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
# 6개의 category
Large_ctg = df['cnt'].groupby(df['Large_category'])  
Large_ctg.sum() 

Large_category
가정/건강         5650
만화              5694
사전/자격증       6212
소설/시/희곡      4774
어린이/청소년     9471
인문/사회/경영    9185
Name: cnt, dtype: int64

## Book Introduction Preprocessing

## data를 X, Y로 분할

In [ ]:
X = df['Introduction']
Y = df['Large_category']

## Y(label) 처리

In [ ]:
# Y값 label Encoding
encoder = LabelEncoder()
labeled_Y = encoder.fit_transform(Y)
label = encoder.classes_
print(label)
print(labeled_Y)

In [ ]:
# encoding mapping 정보를 저장
with open('./data/category_encoder.pickle', 'wb') as f:
  pickle.dump(encoder, f)

In [ ]:
# label을 onehot encoding으로 변환
onehot_Y = to_categorical(labeled_Y)
print(onehot_y)

[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]]
ERROR! Session/line number was not unique in database. History logging moved to new session 366


## X(data) 처리

### 형태소 분석

In [ ]:
okt = Okt()
print('형태소 분석')
for i in range(len(X)):
    X[i] = okt.morphs(X[i])
    if (i % 250 == 0) and (i>1):
        print('.', end='')
    if i % 5000 == 0:
        print('{} / {}'.format(i, len(X)))
print(X)


형태소 분리
ipykernel_launcher:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/kaeun/.conda/envs/tf/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
0 / 40986
....................5000 / 40986
....................10000 / 40986
....................15000 / 40986
....................20000 / 40986
....................25000 / 40986
....................30000 / 40986
....................35000 / 40986
....................40000 / 40986
...0        [내, 몸, 리셋, 다이어트, 10, kg, 밀가루, 단식, 저자, 의, 100일,...   
1        [맛있고, 배부

In [ ]:
print(X.shape)
print(len(X))

(40986,)
40986


### 불용어 제거


In [ ]:
# 불용어 제거
stopwords = pd.read_csv('./data/stopwords.csv')

# 불용어 제거 후 형태소로 이루어진 문장으로 재조합
for i in range(len(X)) :
  result = []
  for j in range(len(X[i])):
    if len(X[i][j]) > 1:         # 길이가 한 글자인 것은 지움
      if X[i][j] not in list(stopwords['stopword']):
        result.append(X[i][j])
  X[i] = ' '.join(result)
  if (i % 250 == 0) and (i>1):
    print('.', end='')
  if i % 5000 == 0:
    print('{} / {}'.format(i, len(X)))
print(X)

ipykernel_launcher:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
0 / 40986
....................5000 / 40986
....................10000 / 40986
....................15000 / 40986
....................20000 / 40986
....................25000 / 40986
....................30000 / 40986
....................35000 / 40986
....................40000 / 40986
...0        리셋 다이어트 10 kg 밀가루 단식 저자 100일 밀가루 단식 다이어트 매주 새로...
1        맛있고 배부른데 빠지는 다이어트 레시피 끝판 101 가지 2018년 2019년 20...   
2        주부 매일 인스타그램 밥상 메뉴 올리는 이유 먹고 사는지 자랑 하려는 절대 엄마 아...
3        가루 큐브 밥솥 칸막이 활용 최초 이유식 엄마 정성 사랑 듬뿍 담긴 이유식 현실 쉽...
4        다이어트 맛있게 먹어야 성공한다 미니 22 kg 엄마 17 kg 감량 수많은 다이어...
                                            ...                                     
40981    인생 쉽지 흘러가지 지성 이성 내린 선택 판단 절대 이지 느꼈을 불확실 성의 시대 ...
40982    인가 욕망 좇는 주도

### 토크나이징

In [ ]:
token = Tokenizer()
token.fit_on_texts(X)                  
tokened_X = token.texts_to_sequences(X)
print(tokened_X[0])

[16483, 993, 180, 5186, 6332, 3991, 24, 8876, 6332, 3991, 993, 5910, 22, 200, 311, 4762, 2440, 129, 40256, 993, 8876, 335, 23, 34, 6243, 129, 2440, 5304, 40256, 993, 6332, 8769, 180, 5186, 5729, 8876, 886, 638, 4827, 1305, 3955, 546, 577, 457, 1510, 870, 57938, 41, 27160, 96180, 23281, 3731, 4863, 1736, 870, 7995, 457, 870, 3391, 47, 2349, 16887, 632, 16269, 8876, 96181, 78, 200, 6269, 864, 7132, 88, 5729, 328, 2267, 1627, 8385, 241, 29379, 6332, 8769, 8386, 1172, 24780, 11517, 441, 3551, 63360, 434, 2677, 125568, 2164, 508, 5350, 3281, 6332, 15119, 632, 3206, 3500, 96182, 296, 53669, 670, 361, 6332, 3991, 6062, 2803, 641, 47, 1474, 125569, 6, 1038, 296, 798, 3433, 1736, 744]


In [ ]:
# token 저장
with open('./data/book_token.pickle', 'wb') as f:
  pickle.dump(token, f)

## data 확인

In [ ]:
# 형태소 개수 확인
wordsize = len(token.word_index) + 1
print('wordsize is : ', wordsize)  # index 0를 padding 으로 추가 예정


wordsize is :  218318


In [ ]:
# tokend_X의 mean 값을 maxlen으로 선정
tokened_len = []
for i in range(len(tokened_X)):
    tokened_len.append(len(tokened_X[i]))

mean = np.mean(tokened_len)
max = int(np.trunc(mean))
print(max)

196


In [ ]:
# # 가장 긴 문장의 길이 확인
# max = 0
# for i in range(len(tokened_X)):
#   if max < len(tokened_X[i]):
#       max = len(tokened_X[i])
      
#       print('max is : ', max) 

In [ ]:
# padding
X_pad = pad_sequences(tokened_X, max)  # 앞쪽을 0으로 채움

print(X_pad[:10])

ERROR! Session/line number was not unique in[[    0     0     0 ...  3433  1736   744]
 [    0     0     0 ... 11441    10 29381]
 [    0     0     0 ...  1247 38591   696]
 ...
 [ 9929 13206 44406 ...   436   684  1016]
 [    0     0     0 ...   125   537  5503]
 [    0     0     0 ... 25882   289  1347]] database. History logging moved to new session 364



## Train, Test set split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_pad, onehot_Y, test_size=0.2)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(32788, 196)
(8198, 196)
(32788, 6)
(8198, 6)


## Train, Test set 저장

In [ ]:
xy = X_train, X_test, Y_train, Y_test
np.save('./data/book_data_max_{}_wordsize_{}'.format(max, wordsize), xy)

/home/kaeun/.conda/envs/tf/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
